In [5]:
import numpy as np
import pandas as pd
import requests
import regex as re
import nltk
import spacy
import requests

from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from xmltodict import parse
from bs4 import BeautifulSoup
from snowballstemmer import TurkishStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from collections import Counter
from xmltodict import parse

import random

# HW3

## Text Processing

### Q1

1. Modify the code I wrote in lecture 8 with what you have learnt in lecture 9 and correctly tokenize the text both on the word and sentence level, and by removing the stopwords. Rewrite the `getSummary` function and all the other functions that it depends by maing these corrections.

2. Rewrite the code I wrote for `getKeywords` function making the same corrections.

3. Test your code from parts 1 and 2 on random articles from the Guardian.

4. Rewrite the `getSubjectGuardian` function for another newspaper in English, and test your code from part 1 and 2 on random articles from this new newspaper.

In [6]:
with requests.get('https://www.theguardian.com/film/rss') as link:
    raw = parse(link.text)
nal = raw['rss']['channel']['item']
def getSubjectGuardian(subject):
    with requests.get(f'https://www.theguardian.com/{subject}/rss') as link:
        raw = parse(link.text)
    return raw['rss']['channel']['item']
nba = getSubjectGuardian('sport/nba')
film = getSubjectGuardian('film')
fashion = getSubjectGuardian('fashion')

In [7]:
nba[0]['link']
with requests.get(nba[0]['link']) as link:
    raw = BeautifulSoup(link.content,'html.parser')
swEN = stopwords.words('english')

Q1.1 : We are rewriting functions with corrections.

In [8]:
def getText(url):
    with requests.get(url) as link:
        raw = BeautifulSoup(link.content,'html.parser')
    return ' '.join([x.text for x in raw.find_all('p')])

In [9]:
def processText(text):
    sentences = {'sentences': sent_tokenize(text)}
    sentences.update({'cleanedSentences': [re.sub(r'[^\p{Letter}\s]','',sentence.lower()) for sentence in sentences['sentences']]})
    return [re.sub(r'[^\w\s]','',x.lower()) for x in sentences['cleanedSentences']]

In [10]:
def getMatrix(sentences):
    vectorizer = CountVectorizer(stop_words=swEN)
    return vectorizer.fit_transform(sentences)

def getSummary(text,k):
    sentences = processText(text)
    matrix = getMatrix(sentences)
    projection = PCA(n_components=1)
    weights = projection.fit_transform(matrix.toarray())
    res = list(zip(weights.transpose()[0],range(112),sentences))
    tmp = sorted(res,key=lambda x: x[0],reverse=True)[:k]
    return sorted(tmp, key=lambda x: x[1])

Q1.2: Tokenized and removed stopwords.

In [11]:
def getKeywords(text,sw,k):
    ex = {'sentences': sent_tokenize(text)}
    ex.update({'cleanedSentences': [re.sub(r'[^\p{Letter}\s]','',sentence.lower()) for sentence in ex['sentences']]})     
    vectorizer = CountVectorizer(stop_words=sw)
    matrix = vectorizer.fit_transform(ex['cleanedSentences'])
    words = vectorizer.get_feature_names()
    projection = PCA(n_components=1)
    tmp = projection.fit_transform(matrix.transpose().toarray())
    weights = tmp.transpose()[0]
    return sorted(zip(weights,words),key=lambda x: x[0], reverse=True)[:k]

Q1.3: Testing our functions.

In [23]:
i = random.randint(1, 10)
text=getText(fashion[i]['link'])

print(getSummary(text,3))
getKeywords(text,swEN,3)

[(3.977206769207988, 0, 'the queer eye presenters look into skin bleaching and colourism sees him sensitively interview kelly rowland listen to abusive tweets  and reveal his own heartrending past when tan france was nine he stole his cousins skin bleaching cream'), (2.45919096152794, 3, 'the shame and guilt that queer eyes resident fashion expert feels about his youthful pursuit of lighter skin is the ostensible motive behind tan france beauty and the bleach bbc two a celebrityodyssey documentary that starts as is now custom with the obligatory suitcasepacking shot'), (2.1929839477887567, 4, 'the doncasterraised stylist is leaving his deluxe utah home and returning to mustyfusty old england to reckon with his feelings about skin bleaching and investigate the wider problem of colourism the idea that fairer complexions are favoured in black and asian communities as well as society at large')]


[(2.3761318802671063, 'skin'),
 (1.8739540399901167, 'bleaching'),
 (1.2256589189685063, 'france')]

In [22]:
i = random.randint(1, 10)
text=getText(nba[i]['link'])

print(getSummary(text,3))
getKeywords(text,swEN,3)

[(0.4596752430077338, 6, 'somebody get me in contact with the young lady'), (0.5297234201070614, 9, 'so thats definitely unacceptable on my part and i take full responsibility'), (4.024848996021901, 12, 'so that was definitely wrong a lot of emotions and hopefully i can get in contact with the young girl and sincerely apologize to her and do something nice for her but thats definitely on me he added that he wouldnt fight any punishment handed down by the nba')]


[(1.7940987603897578, 'definitely'),
 (1.0564636452389606, 'get'),
 (1.0131352889221652, 'contact')]

Q1.4: Testing with different newspaper

In [14]:
def getSubjectNews(subject):
    with requests.get(f'http://en.people.cn/rss/{subject}.xml') as link:
        raw = parse(link.text)
    return raw['rss']['channel']['item']
china = getSubjectNews('China')
business = getSubjectNews('Business')

In [15]:
i = random.randint(1, 10)
text=getText(china[i]['link'])
print(getSummary(text,3))
getKeywords(text,swEN,3)

[(-8.760707733967617, 0, '\n \n\n\n cpc\xa0newschina\xa0tibet\xa0onlineforum \nsina microblog\ntencent microblog\nfeedback\nrss\n editor张茜 \r\n                      \xa0\xa0\xa0\xa0\n 双语词典dictionary besides mars china plans deep space tracking   best space pictures of   giant christmas ornament seen in space in time for holiday  chinas space industry to achieve leapforward development  us astronomers discover buckyballs in space  china to probe deep space  china made  breakthroughs in space telemetry tracking control  light and space create tranquility  interview china contributes to spacebased information access a lot un official  magnificent space photos in june    tibet poised to embrace even brighter future  years after peaceful liberation thunderstorms forecast to continue in north china chinese tourism official confident of crossstrait travel anger over obama meeting dalai lama chinese official calls for more language culture exchanges with foreign countries senior chinese leader

[(7.977861796943847, 'space'),
 (6.884773045137203, 'china'),
 (1.9807812882548177, 'chinese')]

In [16]:
text=getText(business[0]['link'])

print(getSummary(text,3))
getKeywords(text,swEN,3)

[(6.218285461092141, 0, 'set as homepageregistersign inchinesebig frenchrussianspanishjapanesearabic special coveragepd online databasesina microblogtencent microblogfeedback newsopinionschina militaryforeign affairslearn chinesechina studyforum edited and translated by peoples daily online according to data on every countrys gold reserves recently issued by world gold council wgc the united states gold reserves are  tons accounting for  percent of the worlds total and the untied states is still the largest gold reserve country'), (2.2260750504207176, 3, 'in comparison the untied states gold reserves account for  percent of its foreign reserve and even emerging countries including russia and india have gold reserves accounting for more than  percent of their foreign reserves'), (2.9840211287671834, 6, 'the gold purchased by every country in  is three times as much as the gold they purchased in in fact before standard and poors downgraded the us credit rating the central bank of every c

[(9.36416921736123, 'gold'),
 (5.573584647458165, 'reserves'),
 (1.7843895533955678, 'foreign')]

### Q2

Write a function that returns all named entities (proper names, country names, corporation names only) from a URL. Function should take the URL as the input and must return the list of named entities from that URL. Test your code on random articles from the Guardian. Don't use the NLTK's NER that I demonstrated during the lecture. Use the SpaCY's NER function.

We wrote the function with the NER function of the spacy library.

In [17]:
def function(url):
    with requests.get(f'https://www.theguardian.com/sport/nba/rss') as link:
        raw = parse(link.text)
        raw1= raw['rss']['channel']['item']
        i = random.randint(1, 10)
    with requests.get(raw1[i]['link']) as link:
        raw = BeautifulSoup(link.content,'html.parser')
        a=' '.join([x.text for x in raw.find_all('p')])
        ner = spacy.load("en_core_web_sm")
        a1 = ner(a)
        print([i.text for i in a1.ents if i.label_ in ('GPE','ORG','PERSON')])
function(f'https://www.theguardian.com/nba/rss')

['Trae Young', 'Atlanta Hawks', 'Clint Capela', 'the Cleveland Cavaliers', 'the Eastern Conference’s', 'Hawks', 'Cavs', 'Bogdan Bogdanovic', 'Atlanta', 'Miami', 'South Florida', 'Hawks', 'Capela', 'Lauri Markkanen', 'Cleveland', 'Cavs', 'LeBron James', 'James', 'Hawks', 'Cleveland', 'Rocket Mortgage Fieldhouse', 'Cavs', 'Cleveland', 'Cavs', 'Young', 'Cleveland', 'Cavs', 'James', 'NBA', 'Cleveland', 'Cleveland', 'Jarrett Allen', 'Allen', 'Hawks', 'Young', 'Cleveland', 'Atlanta', 'Hawks', 'Markkanen', 'Cleveland', 'Atlanta', 'Brandon Ingram', 'Los Angeles', 'Willie Green', 'New Orleans', 'Green', 'Clipper', 'Zion Williamson', 'Trey Murphy', 'CJ McCollum', 'Larry Nance Jr.', 'New Orleans', 'Nance', 'McCollum', 'Portland', 'Paul George', 'Kawhi Leonard', 'Marcus Morris', 'Reggie Jackson', 'Clippers', 'Pelicans', 'Jackson', 'Morris', 'Robert Covington', 'Ivica Zubac', 'Ingram', 'Pelicans', 'New Orleans', 'McCollum', 'Murphy', 'Murphy', 'Clippers', 'Jackson', 'Morris', 'Clippers', 'Luke Kenn

### Q3

1. Write a function that returns the most positive and the most negative sentences from a text. The function must take the text as the input and must return a 2-tuple: the first element as the most positive and the second as the most negative sentence with their polarity scores.

2. Test your function on random articles from the Guardian.

In [18]:
analyser = SentimentIntensityAnalyzer()

The function first takes the text and splits it into sentences. Then it takes the compound score of each sentence into a list together with the sentence. Then it converts this list as a pandas dataframe and ranks the compound scores from smallest to largest. It takes the largest and smallest data as a new pandas dataframe. It turns that dataframe into a list of 2 tuples.

In [19]:
def textanalyser(text):
    sentences = sent_tokenize(text)
    sentence_scores = []
    for number, sentence in enumerate(sentences):
        scores = analyser.polarity_scores(sentence)
        sentence_scores.append({'sentence': sentence, 'sentence_number': number+1, 'sentiment_score': scores['compound']})
    df = pd.DataFrame(sentence_scores)
    df = df.sort_values(by='sentiment_score')
    df2 = df.iloc[[0, -1]]
    result = list(df2.itertuples(index=False, name=None))
    neg = result[0]
    pos = result[1]
    return neg,pos

In [21]:
i = random.randint(1, 10)
textanalyser(getText(nba[i]['link']))

(('Trae Young scored 32 of his 38 points in the second half and the Atlanta Hawks overcame losing center Clint Capela to a knee injury to beat the Cleveland Cavaliers 107-101 in the play-in on Friday night and earn the Eastern Conference’s No 8 playoff seed.',
  1,
  -0.765),
 ('I’ve been in seasons like that, it’s hard to keep going and keep striving for it.” The Clippers were dealt a huge blow earlier in the day when Paul George entered the league’s health and safety protocols.',
  42,
  0.7351))